In [11]:
import onnx
import onnx_graphsurgeon as gs
import onnxruntime as ort
import torch
import numpy as np
from rich import print
from onnx import TensorProto

## ScatterND using Graph Surgeon 

In [12]:
## updates rank = data rank+indices rank - indices.shape[-1]-1
## output shape same as data shape 

In [13]:
data = gs.Variable("data", dtype=TensorProto.FLOAT, shape=[8])
indices = gs.Variable("indices", dtype=TensorProto.INT64, shape=[4, 1])
updates = gs.Variable("updates", dtype=TensorProto.FLOAT, shape=[4])

scatter_output=gs.Variable("scatter_output",dtype=TensorProto.FLOAT,shape=[8])

In [14]:
scatter_nd= gs.Node(op="ScatterND",name="scatter_node",inputs=[data,indices,updates],outputs=[scatter_output])

graph=gs.Graph(nodes=[scatter_nd],inputs=[data,indices,updates],outputs=[scatter_output],opset=18)

onnx.save(gs.export_onnx(graph),"./models/scatternd.onnx")

## Inputs

In [ ]:

#INPUTS

ort_session=ort.InferenceSession("./models/scatternd.onnx")

input_values =  [np.random.randn(*x.shape).astype(np.float32) if i!=1 else np.random.randint(0,5,x.shape).astype(np.int64) for i,x in enumerate(ort_session.get_inputs())]
input_dict = dict(zip((x.name for x in ort_session.get_inputs()),input_values))

print(input_dict)

d,i,u=input_values

data = torch.from_numpy(d)
indices = torch.from_numpy(i)
updates = torch.from_numpy(u)


{
    'data': array([ 0.37362203, -0.27512082, -0.5601058 , -0.60849917,  1.303471  ,
       -1.4730226 ,  0.8377629 , -0.09678597], dtype=float32),
    'indices': array([[2],
       [3],
       [2],
       [0]]),
    'updates': array([-1.3531327 ,  0.42757785,  2.201315  , -0.6370728 ], dtype=float32)
}

## Replacements

In [ ]:
import torch

class ScatterNDModule(torch.nn.Module):
    def forward(self, data, indices, updates):
        # data: (8,)
        # indices: (M, 1)
        # updates: (M,)
        
        output = data.clone()
        
        M = indices.size(0)
        for i in range(M):
            idx = indices[i, 0]
            part1 = output[0:idx] 
            print("Part1",part1)
            update = updates[i:i+1]
            print("Update",update)
            part2 = output[idx + 1:output.size(0)]
            print("Part2",part2)
            output = torch.cat([part1, update, part2], dim=0)
            print("Output",output)

        return output


if __name__ == "__main__":
    module = ScatterNDModule()
    # output = module(data, indices, updates)
    # print("Updated Output:", output)
    print(input_values)
    
    onnx_path = "./models/final.onnx"
    torch.onnx.export(module, (data, indices, updates), onnx_path,
                    input_names=["data", "indices", "updates"],
                    output_names=["output"],
                    opset_version=17)

    print("ONNX model saved at:", onnx_path)

[
    array([ 0.37362203, -0.27512082, -0.5601058 , -0.60849917,  1.303471  ,
       -1.4730226 ,  0.8377629 , -0.09678597], dtype=float32),
    array([[2],
       [3],
       [2],
       [0]]),
    array([-1.3531327 ,  0.42757785,  2.201315  , -0.6370728 ], dtype=float32)
]

Part1 tensor([ 0.3736, -0.2751])

Update tensor([-1.3531])

Part2 tensor([-0.6085,  1.3035, -1.4730,  0.8378, -0.0968])

Output tensor([ 0.3736, -0.2751, -1.3531, -0.6085,  1.3035, -1.4730,  0.8378, -0.0968])

...

Part1 tensor([])

Update tensor([-0.6371])

Part2 tensor([-0.2751,  2.2013,  0.4276,  1.3035, -1.4730,  0.8378, -0.0968])

Output tensor([-0.6371, -0.2751,  2.2013,  0.4276,  1.3035, -1.4730,  0.8378, -0.0968])

ONNX model saved at: ./models/final.onnx

## ORT Inference

In [17]:

#INPUTS

ort_session=ort.InferenceSession("./models/scatternd.onnx")

input_values =  [np.random.randn(*x.shape).astype(np.float32) if i!=1 else np.random.randint(0,5,x.shape).astype(np.int64) for i,x in enumerate(ort_session.get_inputs())]
input_dict = dict(zip((x.name for x in ort_session.get_inputs()),input_values))

print(input_dict)

d,i,u=input_values

data = torch.from_numpy(d)
indices = torch.from_numpy(i)
updates = torch.from_numpy(u)


ort_session=ort.InferenceSession("./models/scatternd.onnx")
output1=ort_session.run(None,input_dict)
print(output1)


ort_session=ort.InferenceSession("./models/final.onnx")
output=ort_session.run(None,input_dict)
print(output)


{
    'data': array([ 0.54924595, -0.488373  , -0.522974  , -1.842488  ,  0.38043967,
        1.2830617 , -0.8789149 ,  0.6629215 ], dtype=float32),
    'indices': array([[1],
       [0],
       [0],
       [0]]),
    'updates': array([ 1.0100797 , -0.11505555, -1.1076103 ,  0.0081938 ], dtype=float32)
}

[
    array([ 0.0081938 ,  1.0100797 , -0.522974  , -1.842488  ,  0.38043967,
        1.2830617 , -0.8789149 ,  0.6629215 ], dtype=float32)
]

[
    array([ 0.0081938 ,  1.0100797 , -0.522974  , -1.842488  ,  0.38043967,
        1.2830617 , -0.8789149 ,  0.6629215 ], dtype=float32)
]

In [18]:
from sklearn.metrics import mean_squared_error

mean_squared_error(output,output1)

np.float32(0.0)